<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-overview" data-toc-modified-id="Project-overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project overview</a></span></li><li><span><a href="#Scraping-otodom-link-for-offer-references" data-toc-modified-id="Scraping-otodom-link-for-offer-references-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scraping otodom link for offer references</a></span><ul class="toc-item"><li><span><a href="#Understanding-href-structure" data-toc-modified-id="Understanding-href-structure-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Understanding href structure</a></span></li><li><span><a href="#Extracting-all-offers-from-a-page" data-toc-modified-id="Extracting-all-offers-from-a-page-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Extracting all offers from a page</a></span></li><li><span><a href="#Extracting-all-following-pages" data-toc-modified-id="Extracting-all-following-pages-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Extracting all following pages</a></span></li><li><span><a href="#Create-a-list-of-all-offers" data-toc-modified-id="Create-a-list-of-all-offers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Create a list of all offers</a></span></li></ul></li><li><span><a href="#Extract-data-from-single-offer" data-toc-modified-id="Extract-data-from-single-offer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract data from single offer</a></span><ul class="toc-item"><li><span><a href="#Investigate-structure-of-single-offer-page" data-toc-modified-id="Investigate-structure-of-single-offer-page-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Investigate structure of single offer page</a></span></li><li><span><a href="#Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries" data-toc-modified-id="Drill-down-to-data-stored-in-json-and-strat-exploring-key-dictionaries-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Drill down to data stored in json and strat exploring key dictionaries</a></span></li><li><span><a href="#Extract-key-details-into-clear-dictionary" data-toc-modified-id="Extract-key-details-into-clear-dictionary-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Extract key details into clear dictionary</a></span></li></ul></li><li><span><a href="#Combine-offers-into-json-list-and-transform-to-DataFrame" data-toc-modified-id="Combine-offers-into-json-list-and-transform-to-DataFrame-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combine offers into json list and transform to DataFrame</a></span><ul class="toc-item"><li><span><a href="#Combine-offers-data-as-single-json-list" data-toc-modified-id="Combine-offers-data-as-single-json-list-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Combine offers data as single json list</a></span></li><li><span><a href="#Parse-parameters-data" data-toc-modified-id="Parse-parameters-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Parse parameters data</a></span></li><li><span><a href="#Parse-location-data" data-toc-modified-id="Parse-location-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Parse location data</a></span></li><li><span><a href="#Parse-characteristics-data" data-toc-modified-id="Parse-characteristics-data-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Parse characteristics data</a></span></li></ul></li></ul></div>

In [1]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [2]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

# Project overview
<a id="project_overview"><a>

# Scraping otodom link for offer references


In [3]:
link="https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26"

In [4]:
html = urlopen(link)
bs = BeautifulSoup(html, 'lxml')

## Understanding href structure
<a id="href_struct"><a>

In [5]:
for item in bs.findAll("a",href=True):
    print(item)
    print("\n\n")

<a class="navbar-brand" href="https://www.otodom.pl" title="Otodom. Najpopularniejszy serwis nieruchomości">www.otodom.pl</a>



<a aria-expanded="false" class="dropdown-toggle" data-toggle="dropdown" data-tracking="classifieds_click" href="https://www.otodom.pl/sprzedaz/" role="button">Ogłoszenia <span class="caret"></span></a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"sale_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/sprzedaz/">Na sprzedaż</a>



<a data-observed-action="" data-tracking="classifieds_click" data-tracking-data='{"touch_point_button":"rent_dropdown","touch_point_page": "ads"}' href="https://www.otodom.pl/wynajem/">Na wynajem</a>



<a data-observed-action="" data-tracking="developers_click" data-tracking-data='{"touch_point_button":"investments_dropdown", "touch_point_page": "ads"}' href="https://www.otodom.pl/inwestycje/home/">Inwestycje</a>



<a aria-expanded="false" class="dropdown

It seems all offer hrefs contain "www.otodom.pl/oferta/" in their adress. 

The links to move to next page of search results contain "page=" label

In [6]:
bs.findAll("a",href=True)[1].attrs.keys()

dict_keys(['href', 'class', 'data-toggle', 'role', 'aria-expanded', 'data-tracking'])

## Extracting all offers from a page


In [7]:
for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/"))[:10]:
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/oferta/kawalerka-w-centrum-ul-anielewicza-30a-ID45lSw.html#8e0a67b43c


https://www.otodom.pl/oferta/kawalerka-w-centrum-ul-anielewicza-30a-ID45lSw.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-9-pietro-na-ul-karolkowa-28-ID45F5y.html#8e0a67b43c


https://www.otodom.pl/oferta/apartament-9-pietro-na-ul-karolkowa-28-ID45F5y.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-56-5m2-warszawa-praga-poludnie-ID45EwQ.html#8e0a67b43c


https://www.otodom.pl/oferta/bezposrednio-56-5m2-warszawa-praga-poludnie-ID45EwQ.html#8e0a67b43c


https://www.otodom.pl/oferta/przestronna-kawalerka-mozliwe-2-pok-duzy-balkon-ID45Qtc.html#8e0a67b43c


https://www.otodom.pl/oferta/przestronna-kawalerka-mozliwe-2-pok-duzy-balkon-ID45Qtc.html#8e0a67b43c


https://www.otodom.pl/oferta/umeblowany-apartament-z-winda-zen-garden-sadyba-ID45QsY.html#8e0a67b43c


https://www.otodom.pl/oferta/umeblowany-apartament-z-winda-zen-garden-sadyba-ID45QsY.html#8e0a67b43c




In [8]:
def get_page_offers(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    offer_list=[]

    for item in bs.findAll("a", href=re.compile("www.otodom.pl/oferta/")):
        offer_list.append(item.attrs["href"])
    offer_list=list(set(offer_list))
    return(offer_list)

## Extracting all following pages


In [9]:
for item in bs.findAll("a", href=re.compile("page=")):
    print(item.attrs["href"])
    print("\n")

https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=3


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=500


https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=2




In [10]:
def get_next_page(link):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'lxml')
                    
    next_page=bs.findAll("a", href=re.compile("page="))[0].attrs["href"]

    
    return(next_page)

In [11]:
link

'https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26'

In [12]:
base_next_page=get_next_page(link)
base_next_page=base_next_page[:-1]

In [13]:
page_list=[]
page_list.append(link)

In [14]:
page_list

['https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26']

In [15]:
page_max=500
page_list=[link]
for i in range(2,page_max+1):
#    print(str(i))
    new_page=base_next_page+str(i)
#    print(new_page)
    page_list.append(new_page)

In [16]:
len(page_list)

500

## Create a list of all offers 


In [17]:
all_offers=[]
for i in range(0,len(page_list)-1):

    all_offers=all_offers+get_page_offers(page_list[i])
    if i%50==0:
        print("Scraping offers from link")
        print(page_list[i])
        print("Combined number of offers={}\n".format(len(all_offers)))
    
        

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26
Combined number of offers=30

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=51
Combined number of offers=1530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=101
Combined number of offers=3030

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=151
Combined number of offers=4530

Scraping offers from link
https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/?search%5Bregion_id%5D=7&search%5Bsubregion_id%5D=197&search%5Bcity_id%5D=26&page=201
Combined number of offers=6030

Scraping offers from link
https://www

In [18]:
len(all_offers)

14969

# Extract data from single offer

## Investigate structure of single offer page 

In [19]:
offer=all_offers[10]

In [20]:
html = urlopen(offer)
bs = BeautifulSoup(html, 'lxml')

In [21]:
bs.title

<title data-react-helmet="true">2 pokoje, mieszkanie na sprzedaż - Warszawa, Bielany - 60498481 • www.otodom.pl</title>

In [22]:
print(bs.prettify())

<!DOCTYPE html>
<html lang="pl">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="n28MFqvY-A18prHhqojesTsiERCQQdGvoC3I" name="csrf-token"/>
  <title data-react-helmet="true">
   2 pokoje, mieszkanie na sprzedaż - Warszawa, Bielany - 60498481 • www.otodom.pl
  </title>
  <meta content="pl" data-react-helmet="true" http-equiv="Content-Language"/>
  <meta content="index, follow" data-react-helmet="true" name="robots"/>
  <meta content="Świetne 2 pokojowe mieszkanie na sprzedaż w miejscowości Warszawa, Bielany, za cenę 430 000 zł. To mieszkanie na sprzedaż  na 2 piętrze ma 37 m² powierzchni użytkowej i 37 m² powierzchni całkowitej. Właściciel jako najważniejsze zalety mieszkania wymienia: piwnica, winda, drzwi / okna antywłamaniowe. Otodom 60498481" data-react-helmet="true" name="description"/>
  <meta co

Afte investigating, where the specific offer price could be found, we can see that it appears most times within sript with "id"= "server-app-state". Let's parse page html to find this script. 

In [23]:
data_script=bs.find("script",{"id":"server-app-state"})

In [24]:
data_script

<script id="server-app-state" type="application/json">{"initialProps":{"meta":{"target":{"Area":"37","AreaRange":[],"Build_year":"1967","Building_floors_num":"10","Building_material":["concrete_plate"],"Building_ownership":["full_ownership"],"Building_type":["block"],"City":"warszawa","City_id":"26","Construction_status":["to_renovation"],"Country":"Polska","Extras_types":["basement","lift"],"Floor_no":["floor_2"],"Heating":["urban"],"Id":"60498481","MarketType":"secondary","OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9hcG9sbG8taXJlbGFuZC5ha2FtYWl6ZWQubmV0L3YxL2ZpbGVzL2V5Sm1iaUk2SW1sbk1URmpjWGRoY21wcWJ6RXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLjl0RVoyTVh0RXFmMFNBckVwekVxa3dKeDF3dFV0eFhRN3dWalRLN1JSRzgvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":430000,"PriceRange":["250001-500000"],"Price_per_m":11622,"ProperType":"mieszkanie","Province":"mazowieckie","RegularUser":"y","Rent":350,"Rooms_num":["2"],"Security_types

In [25]:
data_script.contents[0]

'{"initialProps":{"meta":{"target":{"Area":"37","AreaRange":[],"Build_year":"1967","Building_floors_num":"10","Building_material":["concrete_plate"],"Building_ownership":["full_ownership"],"Building_type":["block"],"City":"warszawa","City_id":"26","Construction_status":["to_renovation"],"Country":"Polska","Extras_types":["basement","lift"],"Floor_no":["floor_2"],"Heating":["urban"],"Id":"60498481","MarketType":"secondary","OfferType":"sprzedaz","Photo":"aHR0cHM6Ly9hcG9sbG8taXJlbGFuZC5ha2FtYWl6ZWQubmV0L3YxL2ZpbGVzL2V5Sm1iaUk2SW1sbk1URmpjWGRoY21wcWJ6RXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLjl0RVoyTVh0RXFmMFNBckVwekVxa3dKeDF3dFV0eFhRN3dWalRLN1JSRzgvaW1hZ2U7cz02NTV4NDkxO3E9ODA=","Price":430000,"PriceRange":["250001-500000"],"Price_per_m":11622,"ProperType":"mieszkanie","Province":"mazowieckie","RegularUser":"y","Rent":350,"Rooms_num":["2"],"Security_types":["anti_burglary_door","entryphone","monitoring"],"S

## Drill down to data stored in json and strat exploring key dictionaries

In [26]:
data_json=json.loads(data_script.contents[0])

In [27]:
data_json

{'initialProps': {'meta': {'target': {'Area': '37',
    'AreaRange': [],
    'Build_year': '1967',
    'Building_floors_num': '10',
    'Building_material': ['concrete_plate'],
    'Building_ownership': ['full_ownership'],
    'Building_type': ['block'],
    'City': 'warszawa',
    'City_id': '26',
    'Construction_status': ['to_renovation'],
    'Country': 'Polska',
    'Extras_types': ['basement', 'lift'],
    'Floor_no': ['floor_2'],
    'Heating': ['urban'],
    'Id': '60498481',
    'MarketType': 'secondary',
    'OfferType': 'sprzedaz',
    'Photo': 'aHR0cHM6Ly9hcG9sbG8taXJlbGFuZC5ha2FtYWl6ZWQubmV0L3YxL2ZpbGVzL2V5Sm1iaUk2SW1sbk1URmpjWGRoY21wcWJ6RXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLjl0RVoyTVh0RXFmMFNBckVwekVxa3dKeDF3dFV0eFhRN3dWalRLN1JSRzgvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
    'Price': 430000,
    'PriceRange': ['250001-500000'],
    'Price_per_m': 11622,
    'ProperType': 'mieszkanie',
    'Prov

After parsing json dictionary from the chosen sript, we can start to see clear patters. Let's focus on dictionary embedded within ['initialProps'], as it seems to have all the data we need to extract 

In [28]:
#parameters dict
data_json['initialProps']["meta"]["target"]

{'Area': '37',
 'AreaRange': [],
 'Build_year': '1967',
 'Building_floors_num': '10',
 'Building_material': ['concrete_plate'],
 'Building_ownership': ['full_ownership'],
 'Building_type': ['block'],
 'City': 'warszawa',
 'City_id': '26',
 'Construction_status': ['to_renovation'],
 'Country': 'Polska',
 'Extras_types': ['basement', 'lift'],
 'Floor_no': ['floor_2'],
 'Heating': ['urban'],
 'Id': '60498481',
 'MarketType': 'secondary',
 'OfferType': 'sprzedaz',
 'Photo': 'aHR0cHM6Ly9hcG9sbG8taXJlbGFuZC5ha2FtYWl6ZWQubmV0L3YxL2ZpbGVzL2V5Sm1iaUk2SW1sbk1URmpjWGRoY21wcWJ6RXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLjl0RVoyTVh0RXFmMFNBckVwekVxa3dKeDF3dFV0eFhRN3dWalRLN1JSRzgvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
 'Price': 430000,
 'PriceRange': ['250001-500000'],
 'Price_per_m': 11622,
 'ProperType': 'mieszkanie',
 'Province': 'mazowieckie',
 'RegularUser': 'y',
 'Rent': 350,
 'Rooms_num': ['2'],
 'Security_types': ['anti

In [29]:
data_json['initialProps']["data"]["advert"]["price"]

{'human_value': '430 000 zł',
 'label': 'Cena',
 'unit': 'PLN',
 'value': '430000'}

In [30]:
#Building characteristics data
data_json['initialProps']["data"]["advert"]['characteristics']

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '37',
  'value_translated': '37 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '2',
  'value_translated': '2'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'secondary',
  'value_translated': 'wtórny'},
 {'key': 'building_type',
  'currency': '',
  'label': 'Rodzaj zabudowy',
  'value': 'block',
  'value_translated': 'blok'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_2',
  'value_translated': '2'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '10',
  'value_translated': '10'},
 {'key': 'building_material',
  'currency': '',
  'label': 'Materiał budynku',
  'value': 'concrete_plate',
  'value_translated': 'wielka płyta'},
 {'key': 'windows_type',
  'currency': '',
  'label': 'Okna',
  'value': 'plastic',
  'value_translated': 'plastikowe'},
 {'key': 'heating',
  'currency': '',
  

In [31]:
# location data
data_json["initialProps"]["data"]["advert"]['location']

{'address': 'Warszawa, Bielany, ul. Wrzeciono',
 'coordinates': {'latitude': 52.2909864,
  'longitude': 20.9400134,
  'radius': 0,
  'zoom_level': 16},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '38', 'label': 'Bielany', 'type': 'district'}]}

In [32]:
offer_dict={}

In [33]:
offer_dict["params"]=data_json['initialProps']["meta"]["target"]
offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']

## Extract key details into clear dictionary

In [34]:
def get_offer_json(offer):
    html = urlopen(offer)
    bs = BeautifulSoup(html, 'lxml')
    data_script=bs.find("script",{"id":"server-app-state"})
    data_json=json.loads(data_script.contents[0])
    
    offer_dict={}
    offer_dict["params"]=data_json['initialProps']["meta"]["target"]
#    offer_dict["params"]["currency"]=data_json['initialProps']["data"]["advert"]["price"]["unit"]
    offer_dict["location"]=data_json["initialProps"]["data"]["advert"]['location']
    offer_dict["characteristic"]=data_json['initialProps']["data"]["advert"]['characteristics']
    
    return(offer_dict)

In [35]:
get_offer_json(offer)

{'params': {'Area': '37',
  'AreaRange': [],
  'Build_year': '1967',
  'Building_floors_num': '10',
  'Building_material': ['concrete_plate'],
  'Building_ownership': ['full_ownership'],
  'Building_type': ['block'],
  'City': 'warszawa',
  'City_id': '26',
  'Construction_status': ['to_renovation'],
  'Country': 'Polska',
  'Extras_types': ['basement', 'lift'],
  'Floor_no': ['floor_2'],
  'Heating': ['urban'],
  'Id': '60498481',
  'MarketType': 'secondary',
  'OfferType': 'sprzedaz',
  'Photo': 'aHR0cHM6Ly9hcG9sbG8taXJlbGFuZC5ha2FtYWl6ZWQubmV0L3YxL2ZpbGVzL2V5Sm1iaUk2SW1sbk1URmpjWGRoY21wcWJ6RXRRVkJNSWl3aWR5STZXM3NpWm00aU9pSnFNV296YnpFemJUWmlaMjR4TFVGUVRDSXNJbk1pT2lJeE5DSXNJbkFpT2lJeE1Dd3RNVEFpTENKaElqb2lNQ0o5WFgwLjl0RVoyTVh0RXFmMFNBckVwekVxa3dKeDF3dFV0eFhRN3dWalRLN1JSRzgvaW1hZ2U7cz02NTV4NDkxO3E9ODA=',
  'Price': 430000,
  'PriceRange': ['250001-500000'],
  'Price_per_m': 11622,
  'ProperType': 'mieszkanie',
  'Province': 'mazowieckie',
  'RegularUser': 'y',
  'Rent': 350,
  'Rooms_nu

# Combine offers into json list and transform to DataFrame

## Combine offers data as single json list

In [36]:
def create_offers_dict(offers_list):
    combined_offers_list=[]
    for i in range(0, len(offers_list)):
        offer=offers_list[i]
        offer_data={}
        try:
            offer_json=get_offer_json(offer)
            offer_data["Id"]=offer_json["params"]["Id"]
            offer_data["data"]=offer_json
            combined_offers_list.append(offer_data)
        except:
            pass
      
        if i%500==0:
            print("Extracted {}/{} offers at {}".format(i,len(offers_list),str(datetime.now())[:-7]))
    return(combined_offers_list)
        
    

Let's extract all 15k offers - recommend leaving it running for the night as it might take a few hours. 

In [37]:
offers_json=create_offers_dict(all_offers)

Extracted 0/14969 offers at 2020-05-14 20:50:09
Extracted 500/14969 offers at 2020-05-14 20:57:36
Extracted 1000/14969 offers at 2020-05-14 21:04:48
Extracted 1500/14969 offers at 2020-05-14 21:11:41
Extracted 2000/14969 offers at 2020-05-14 21:18:44
Extracted 2500/14969 offers at 2020-05-14 21:25:51
Extracted 3000/14969 offers at 2020-05-14 21:33:01
Extracted 3500/14969 offers at 2020-05-14 21:40:00
Extracted 4000/14969 offers at 2020-05-14 21:46:50
Extracted 4500/14969 offers at 2020-05-14 21:53:52
Extracted 5000/14969 offers at 2020-05-14 22:00:51
Extracted 5500/14969 offers at 2020-05-14 22:07:48
Extracted 6000/14969 offers at 2020-05-14 22:15:12
Extracted 6500/14969 offers at 2020-05-14 22:21:59
Extracted 7000/14969 offers at 2020-05-14 22:28:44
Extracted 7500/14969 offers at 2020-05-14 22:35:40
Extracted 8000/14969 offers at 2020-05-14 22:42:52
Extracted 8500/14969 offers at 2020-05-14 22:50:07
Extracted 9000/14969 offers at 2020-05-14 22:57:10
Extracted 9500/14969 offers at 2020

In [38]:
len(offers_json)

14922

In [39]:
with open('json_dict.txt', 'w') as file:
     file.write(json.dumps(offers_json))

40732231

## Parse parameters data

In [40]:
def parse_param_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_row=json_list[i]["data"]["params"]
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [41]:
df_params=parse_param_data(offers_json)
df_params.drop_duplicates(subset="Id",inplace=True)

In [42]:
df_params.head()

,Area,AreaRange,Area_from,Area_to,Build_year,Building_floors_num,Building_material,Building_ownership,Building_type,City,...,Rent,Rooms_num,Security_types,State,Subregion,Title,Windows_type,env,seller_id,user_type
0,46.16,[],NaN,NaN,1951.0,3.0,[brick],[full_ownership],[block],warszawa,...,NaN,[2],NaN,NaN,powiat-warszawa,RHdhIHBva29qZSB6IG9kZHppZWxuxIUga3VjaG5pxIUsYm...,[plastic],production,NaN,private
1,41.00,[],NaN,NaN,2013.0,4.0,NaN,[full_ownership],[block],warszawa,...,460.0,[2],"[closed_area, entryphone, monitoring]",NaN,powiat-warszawa,bWllc3prYW5pZSAyIHBva29qZSBCaWHFgm/FgsSZa2EgdW...,[plastic],production,NaN,private
2,NaN,NaN,25.0,50.0,NaN,NaN,NaN,NaN,NaN,warszawa,...,NaN,NaN,NaN,[in_building],powiat-warszawa,QXBhcnRhbWVudHkgSU5XRVNUWUNZSk5FIG5hIFByYWR6ZS...,NaN,production,7229179.0,professional
3,95.00,[],NaN,NaN,1951.0,3.0,[brick],[full_ownership],[tenement],warszawa,...,NaN,[4],[entryphone],NaN,powiat-warszawa,V2Fyc3phd2EgTW9rb3TDs3csIFN0YXJ5IE1va290w7N3LC...,[plastic],production,1019317.0,professional
4,140.00,[],NaN,NaN,2002.0,9.0,[brick],[full_ownership],[apartment],warszawa,...,1700.0,[4],[monitoring],NaN,powiat-warszawa,THVrc3VzLCBrb21mb3J0IGkgYmV6cGllY3plxYRzdHdv,NaN,production,742761.0,professional


In [43]:
df_params.columns

Index(['Area', 'AreaRange', 'Area_from', 'Area_to', 'Build_year',
       'Building_floors_num', 'Building_material', 'Building_ownership',
       'Building_type', 'City', 'City_id', 'Construction_status', 'Country',
       'Equipment_types', 'Extras_types', 'Floor_no', 'Floors_num', 'Heating',
       'Id', 'MarketType', 'Media_types', 'OfferType', 'Offered_estates_type',
       'Photo', 'Price', 'PriceRange', 'Price_from', 'Price_per_m',
       'Price_per_m_from', 'Price_to', 'ProperType', 'Province', 'RegularUser',
       'Rent', 'Rooms_num', 'Security_types', 'State', 'Subregion', 'Title',
       'Windows_type', 'env', 'seller_id', 'user_type'],
      dtype='object')

In [62]:
df_params.shape

(12135, 43)

## Parse location data

In [44]:
json_str=offers_json[0]["data"]["location"]

In [45]:
json_str

{'address': 'Warszawa, Wola, Młynów, ul. Leszno',
 'coordinates': {'latitude': 52.238238,
  'longitude': 20.9785314,
  'radius': 0,
  'zoom_level': 16},
 'geoLevel': [{'id': '7', 'label': 'mazowieckie', 'type': 'region'},
  {'id': '197', 'label': 'Warszawa', 'type': 'sub-region'},
  {'id': '26', 'label': 'Warszawa', 'type': 'city'},
  {'id': '963', 'label': 'Młynów', 'type': 'district'}]}

In [46]:
type(json_str["geoLevel"])==list

True

In [47]:
json_str.keys()

dict_keys(['address', 'coordinates', 'geoLevel'])

In [48]:
json_str["coordinates"]['latitude']

52.238238

In [49]:
def extract_location_data(json_entry):
    new_dict={}
    json_str=json_entry["data"]["location"]
    for key in json_str.keys():
        val_level1=json_str[key]
        if type(val_level1)==dict:
            for subkey in val_level1.keys():
                val_level2=val_level1[subkey]
                new_dict[subkey]=val_level2

        elif type(val_level1)==list:
            for i in range(0, len(val_level1)):
                val_level2=val_level1[i]
                new_dict[val_level2["type"]]=val_level2["label"]
        else:

            new_dict[key]=json_str[key]
            
    return(new_dict)


In [50]:
extract_location_data(offers_json[0])

{'address': 'Warszawa, Wola, Młynów, ul. Leszno',
 'latitude': 52.238238,
 'longitude': 20.9785314,
 'radius': 0,
 'zoom_level': 16,
 'region': 'mazowieckie',
 'sub-region': 'Warszawa',
 'city': 'Warszawa',
 'district': 'Młynów'}

In [51]:
def parse_location_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_Id=json_list[i]["Id"]
        parsed_row=extract_location_data(json_list[i])
        parsed_row["Id"]=parsed_Id
        param_list.append(parsed_row)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [52]:
df_location=parse_location_data(offers_json)
df_location.drop_duplicates(subset="Id",inplace=True)

In [53]:
df_location.head()

,,Id,address,city,district,latitude,longitude,radius,region,sub-region,zoom_level
0,NaN,60434297,"Warszawa, Wola, Młynów, ul. Leszno",Warszawa,Młynów,52.238238,20.978531,0,mazowieckie,Warszawa,16
1,NaN,60498475,"Warszawa, Białołęka",Warszawa,Białołęka,52.345895,20.943476,100,mazowieckie,Warszawa,16
2,NaN,60194413,"Warszawa, Praga-Południe",Warszawa,Praga-Południe,52.241680,21.084060,100,mazowieckie,Warszawa,12
3,NaN,59392882,"Warszawa, Mokotów, Górny Mokotów, Różana",Warszawa,Górny Mokotów,52.202488,21.017253,0,mazowieckie,Warszawa,15
4,NaN,60498567,"Warszawa, Żoliborz, Bitwy pod Rokitną",Warszawa,Żoliborz,52.262000,20.986000,0,mazowieckie,Warszawa,15


In [54]:
df_location.columns

Index(['', 'Id', 'address', 'city', 'district', 'latitude', 'longitude',
       'radius', 'region', 'sub-region', 'zoom_level'],
      dtype='object')

In [63]:
df_location.shape

(12135, 11)

## Parse characteristics data

In [55]:
json_str=offers_json[0]["data"]["characteristic"]

In [56]:
json_str

[{'key': 'm',
  'currency': '',
  'label': 'Powierzchnia',
  'value': '46.16',
  'value_translated': '46,16 m²'},
 {'key': 'rooms_num',
  'currency': '',
  'label': 'Liczba pokoi',
  'value': '2',
  'value_translated': '2'},
 {'key': 'market',
  'currency': '',
  'label': 'Rynek',
  'value': 'secondary',
  'value_translated': 'wtórny'},
 {'key': 'building_type',
  'currency': '',
  'label': 'Rodzaj zabudowy',
  'value': 'block',
  'value_translated': 'blok'},
 {'key': 'floor_no',
  'currency': '',
  'label': 'Piętro',
  'value': 'floor_2',
  'value_translated': '2'},
 {'key': 'building_floors_num',
  'currency': '',
  'label': 'Liczba pięter',
  'value': '3',
  'value_translated': '3'},
 {'key': 'building_material',
  'currency': '',
  'label': 'Materiał budynku',
  'value': 'brick',
  'value_translated': 'cegła'},
 {'key': 'windows_type',
  'currency': '',
  'label': 'Okna',
  'value': 'plastic',
  'value_translated': 'plastikowe'},
 {'key': 'heating',
  'currency': '',
  'label': 'Og

In [57]:
def parse_characteristic_data(json_list):
    param_list=[]
    for i in range(0, len(json_list)):
        parsed_id=json_list[i]["Id"]
        extracted_dict={}
        parsed_dict=json_list[i]["data"]["characteristic"]
        for j in range(0, len(parsed_dict)):
            parsed_row=parsed_dict[j]
            extracted_dict[parsed_row["key"]]=parsed_row["value"]
        
        extracted_dict["Id"]=parsed_id
            
            
        
        param_list.append(extracted_dict)
    
    param_json=json.dumps(param_list)
    df_param=pd.read_json(param_json, orient='Id')
    return(df_param)
  

In [58]:
df_characteristic=parse_characteristic_data(offers_json)
df_characteristic.drop_duplicates(subset="Id",inplace=True)

In [59]:
df_characteristic.columns

Index(['Id', 'area_from', 'area_to', 'build_year', 'building_floors_num',
       'building_material', 'building_ownership', 'building_type',
       'construction_status', 'floor_no', 'floors_num', 'free_from', 'heating',
       'm', 'market', 'offered_estates_type', 'price_from', 'price_per_m_from',
       'price_to', 'rent', 'rooms_num', 'state', 'windows_type'],
      dtype='object')

In [60]:
df_characteristic.head()

,Id,area_from,area_to,build_year,building_floors_num,building_material,building_ownership,building_type,construction_status,floor_no,...,m,market,offered_estates_type,price_from,price_per_m_from,price_to,rent,rooms_num,state,windows_type
0,60434297,NaN,NaN,1951.0,3.0,brick,full_ownership,block,ready_to_use,floor_2,...,46.16,secondary,NaN,NaN,NaN,NaN,NaN,2,NaN,plastic
1,60498475,NaN,NaN,2013.0,4.0,NaN,full_ownership,block,ready_to_use,floor_1,...,41.00,secondary,NaN,NaN,NaN,NaN,460.0,2,NaN,plastic
2,60194413,25.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,flats,NaN,7350.0,NaN,NaN,NaN,in_building,NaN
3,59392882,NaN,NaN,1951.0,3.0,brick,full_ownership,tenement,ready_to_use,floor_3,...,95.00,secondary,NaN,NaN,NaN,NaN,NaN,4,NaN,plastic
4,60498567,NaN,NaN,2002.0,9.0,brick,full_ownership,apartment,ready_to_use,floor_2,...,140.00,secondary,NaN,NaN,NaN,NaN,1700.0,4,NaN,NaN


In [64]:
df_characteristic.shape

(12135, 23)

In [61]:
df_params.to_excel("params.xlsx")
df_location.to_excel("location.xlsx")
df_characteristic.to_excel("characteristic.xlsx")